In [1]:
import os
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from torch.utils.data import Dataset, DataLoader

import numpy as np
from PIL import Image

In [2]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 10
no_cuda = False
log_interval = 100

# Model

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Preprocess

In [4]:
train_paths = glob('C:/Users/신상훈/Desktop/새 폴더/dataset/mnist_png/training/*/*.png')[:1000]
test_paths = glob('C:/Users/신상훈/Desktop/새 폴더/dataset/mnist_png/testing/*/*.png')[:1000]

len(train_paths), len(test_paths)

(1000, 1000)

In [5]:
class Dataset(Dataset):
    def __init__(self, data_paths, transform=None):

        self.data_paths = data_paths
        self.transform = transform

    def __len__(self):
        return len(self.data_paths)

    def __getitem__(self, idx):
        path = self.data_paths[idx]
        image = Image.open(path).convert("L")
        label = int(path.split('\\')[-2])
        
        if self.transform:
            image = self.transform(image)

        return image, label

In [6]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    Dataset(train_paths, 
            transforms.Compose([
                transforms.RandomHorizontalFlip(), 
                transforms.ToTensor(), 
                transforms.Normalize(
                    mean=[0.406], 
                    std=[0.225])])
           ),
    batch_size=batch_size, 
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    Dataset(test_paths,
           transforms.Compose([
               transforms.ToTensor(), 
               transforms.Normalize(
                   mean=[0.406], 
                   std=[0.225])])
           ),
    batch_size=batch_size, 
    shuffle=False
)

In [7]:
for i, data in enumerate(train_loader):
    if i == 0:
        print(data[0].shape, data[1].shape)
        break

torch.Size([64, 1, 28, 28]) torch.Size([64])


# Optimization

In [8]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

# Learning Rate Scheduler

In [9]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [10]:
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=0, verbose=True)

# Training

In [12]:
import torchvision
from torch.utils.tensorboard import SummaryWriter

In [13]:
writer = SummaryWriter()

In [17]:
for epoch in range(1, epochs + 1):
    # Train Mode
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)  # https://pytorch.org/docs/stable/nn.html#nll-loss
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
    
    
    # Test mode
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    
    accuracy = 100. * correct / len(test_loader.dataset)
    
    scheduler.step(accuracy, epoch)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))
    if epoch ==0:
        grid = torchvision.utils.make_grid(data)
        writer.add_image('images',grid,epoch)
        writer.add_graph(model,data)
    writer.add_scalar('Loss/train/',loss,epoch)
    writer.add_scalar('Loss/test/',test_loss ,epoch)
    writer.add_scalar('accuracy/test/',accuracy ,epoch)

Train Epoch: 1 [0/1000 (0%)]	Loss: 0.059764

Test set: Average loss: 0.1304, Accuracy: 980/1000 (98%)

Train Epoch: 2 [0/1000 (0%)]	Loss: 0.066617

Test set: Average loss: 0.1304, Accuracy: 980/1000 (98%)

Train Epoch: 3 [0/1000 (0%)]	Loss: 0.061564

Test set: Average loss: 0.1304, Accuracy: 980/1000 (98%)

Train Epoch: 4 [0/1000 (0%)]	Loss: 0.061077

Test set: Average loss: 0.1304, Accuracy: 980/1000 (98%)

Train Epoch: 5 [0/1000 (0%)]	Loss: 0.059673

Test set: Average loss: 0.1304, Accuracy: 980/1000 (98%)

Train Epoch: 6 [0/1000 (0%)]	Loss: 0.067097

Test set: Average loss: 0.1304, Accuracy: 980/1000 (98%)

Train Epoch: 7 [0/1000 (0%)]	Loss: 0.061144

Test set: Average loss: 0.1304, Accuracy: 980/1000 (98%)

Train Epoch: 8 [0/1000 (0%)]	Loss: 0.062371

Test set: Average loss: 0.1304, Accuracy: 980/1000 (98%)

Train Epoch: 9 [0/1000 (0%)]	Loss: 0.062909

Test set: Average loss: 0.1304, Accuracy: 980/1000 (98%)

Train Epoch: 10 [0/1000 (0%)]	Loss: 0.064773

Test set: Average loss: 0.1

# Save Model

In [18]:
save_path = 'model_weight.pt'

In [19]:
torch.save(model.state_dict(),save_path)

In [21]:
model.state_dict()['conv1.weight'].shape

torch.Size([20, 1, 5, 5])

In [22]:
model = Net().to(device)

In [23]:
weight_dict = torch.load('model_weight.pt')

In [24]:
weight_dict

OrderedDict([('conv1.weight',
              tensor([[[[-0.1424, -0.1997,  0.0005,  0.1802,  0.1808],
                        [-0.1228, -0.0365, -0.1675, -0.1557, -0.0544],
                        [ 0.1942, -0.0518,  0.0126,  0.1893,  0.0361],
                        [ 0.1618,  0.1341,  0.1897,  0.0360, -0.1614],
                        [-0.1720, -0.1022,  0.1673,  0.1169,  0.0592]]],
              
              
                      [[[-0.1542, -0.1484,  0.1634, -0.0202, -0.1990],
                        [-0.0267, -0.0491,  0.0033,  0.0847,  0.0777],
                        [-0.1548, -0.0027, -0.0984,  0.1367,  0.1003],
                        [ 0.1088,  0.1818,  0.0993, -0.1985,  0.0516],
                        [-0.1378,  0.0267,  0.0028,  0.1994, -0.1360]]],
              
              
                      [[[-0.1973, -0.0544,  0.0323,  0.1477,  0.1638],
                        [ 0.1526,  0.0621, -0.0573, -0.0408, -0.1071],
                        [ 0.0374, -0.1406,  0.0684,  0

In [25]:
weight_dict.keys()

odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])

In [26]:
model.load_state_dict(weight_dict)

<All keys matched successfully>

In [27]:
model.eval()

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

# Save Entire model

In [28]:
save_path = 'model.pt'

In [30]:
torch.save(model, save_path)

# Load Model

In [33]:
model = torch.load(save_path)

In [34]:
model.eval

<bound method Module.eval of Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)>

# Save, Load and Resuming Training

In [35]:
checkpoint_path = 'checkpint.pt'

In [36]:
torch.save({
    'epoch' : epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss
},checkpoint_path)

In [37]:
model = Net().to(device)

In [38]:
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [40]:
checkpoint = torch.load(checkpoint_path)

In [41]:
checkpoint.keys()

dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'loss'])

In [43]:
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [44]:
model

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [45]:
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 1.0000000000000004e-08
    momentum: 0.5
    nesterov: False
    weight_decay: 0
)

In [46]:
epoch

10

In [47]:
loss

tensor(0.0620, requires_grad=True)

In [48]:
model.train

<bound method Module.train of Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)>